# Load SageMaker GroundTruth annotation

## Load task manifest file mapping text sample IDs to text

In [ ]:
from dataclasses import dataclass
import json
from pathlib import Path
from typing import Dict, List

In [ ]:
manifest_path = Path("annotations/glue-dir-kbase-dev-sagemaker-ground-truth-labeling-clone/annotations/intermediate/1/annotations.manifest")

In [ ]:
assert manifest_path.is_file()

In [ ]:
index_to_text_raw = {}

In [ ]:
with open(manifest_path, "rt") as fin:
    for line in fin:
        columns = line.split("\t")
        index = int(columns[0])
        text = "\t".join(columns[1:-1])
        _ = columns[-1]  # no idea what this column is
        assert index not in index_to_text_raw
        index_to_text_raw[index] = text

## Load annotation for given sample text IDs 

In [ ]:
annotations_dir = Path("annotations/glue-dir-kbase-dev-sagemaker-ground-truth-labeling-clone/annotations/worker-response/iteration-1/")

In [ ]:
assert annotations_dir.is_dir()

In [ ]:
index_to_annotation_raw = {}

In [ ]:
for annotations_subdir in annotations_dir.iterdir():
    index = int(annotations_subdir.name)
    for i, annotations_file in enumerate(annotations_subdir.iterdir()):
        assert i == 0, f"found more than one annotation in {annotations_subdir}"
        with open(annotations_file, "rt") as fin:
            j = json.load(fin)
        assert index not in index_to_annotation_raw
        answers = j["answers"]
        assert len(answers) == 1
        entities = answers[0]['answerContent']['crowd-entity-annotation']['entities']
        index_to_annotation_raw[index] = entities

## Merge text and annotation whole removing episode IDs from text and adjusting entity positions

In [ ]:
@dataclass
class Doc:
    id_: str
    text: str
    annotations: List[Dict]

In [ ]:
for index, annotation_raw in index_to_annotation_raw.items():
    text_raw = index_to_text_raw[index]
    pass

In [ ]:
text_raw

In [ ]:
text_raw[131:137]

In [ ]:
annotation_raw

TODO:

- remove episode ID
- adjust text and tag positions, remember converting escape to single chars
- load into spacy
- visualize for sanity checking